In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
import time

In [3]:
org_df_policy = pd.read_csv ('usa-covid-policy/data/OxCGRT_US_latest.csv')
org_jhu_deaths_df = pd.read_csv('csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
org_jhu_cases_df = pd.read_csv('csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')

/home/nick/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Oxford Policy Data

In [4]:
org_df_policy

CountryName CountryCode RegionName RegionCode Jurisdiction      Date  \
0      United States         USA        NaN        NaN      NAT_GOV  20200101   
1      United States         USA        NaN        NaN      NAT_GOV  20200101   
2      United States         USA        NaN        NaN      NAT_GOV  20200102   
3      United States         USA        NaN        NaN      NAT_GOV  20200102   
4      United States         USA        NaN        NaN      NAT_GOV  20200103   
...              ...         ...        ...        ...          ...       ...   
16249  United States         USA    Wyoming      US_WY    STATE_ALL  20201023   
16250  United States         USA    Wyoming      US_WY    STATE_ALL  20201024   
16251  United States         USA    Wyoming      US_WY    STATE_ALL  20201025   
16252  United States         USA    Wyoming      US_WY    STATE_ALL  20201026   
16253  United States         USA    Wyoming      US_WY    STATE_ALL  20201027   

       C1_School closing  C1_Flag C1_Notes  C2_Workplace closing  ...  \
0                    0.0      NaN      NaN                   0.0  ...   
1                    NaN      NaN      NaN                   NaN  ...   
2                    0.0      NaN      NaN                   0.0  ...   
3                    NaN      NaN      NaN                   NaN  ...   
4                    0.0      NaN      NaN                   0.0  ...   
...                  ...      ...      ...                   ...  ...   
16249                NaN      NaN      NaN                   NaN  ...   
16250                NaN      NaN      NaN                   NaN  ...   
16251                NaN      NaN      NaN                   NaN  ...   
16252                NaN      NaN      NaN                   NaN  ...   
16253                NaN      NaN      NaN                   NaN  ...   

       StringencyIndex StringencyIndexForDisplay  StringencyLegacyIndex  \
0                  0.0                      0.00                    0.0   
1                  NaN                       NaN                    NaN   
2                  0.0                      0.00                    0.0   
3                  NaN                       NaN                    NaN   
4                  0.0                      0.00                    0.0   
...                ...                       ...                    ...   
16249              NaN                     40.74                    NaN   
16250              NaN                     40.74                    NaN   
16251              NaN                     40.74                    NaN   
16252              NaN                     40.74                    NaN   
16253              NaN                     40.74                    NaN   

       StringencyLegacyIndexForDisplay GovernmentResponseIndex  \
0                                 0.00                     0.0   
1                                  NaN                     NaN   
2                                 0.00                     0.0   
3                                  NaN                     NaN   
4                                 0.00                     0.0   
...                                ...                     ...   
16249                            49.52                     NaN   
16250                            49.52                     NaN   
16251                            49.52                     NaN   
16252                            49.52                     NaN   
16253                            49.52                     NaN   

       GovernmentResponseIndexForDisplay  ContainmentHealthIndex  \
0                                   0.00                     0.0   
1                                    NaN                     NaN   
2                                   0.00                     0.0   
3                                    NaN                     NaN   
4                                   0.00                     0.0   
...                                  ...                 

In [5]:
df_policy = org_df_policy[["Date", "RegionCode", "RegionName", "CountryCode", "Jurisdiction", "StringencyIndexForDisplay",
                           "GovernmentResponseIndexForDisplay", "ContainmentHealthIndexForDisplay",
                           "EconomicSupportIndexForDisplay"]]

In [6]:
df_policy.Jurisdiction.unique()
# Removing rows where the Jurisdiction is NAT_GOV since I only want to look at the states
nat_gov_jur = df_policy[df_policy["Jurisdiction"] == "NAT_GOV"].index
df_policy.drop(nat_gov_jur, inplace=True)

/home/nick/.local/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [7]:
# Normalize the Indicies Columns using the max for each col.
indices = df_policy.filter(regex="Index")
for col in indices.columns:
    df_policy[col] /= 100

# Each index is composed of a series of individual policy response indicators. For each
# indicator, we create a score by taking the ordinal value and adding an extra half-point
# if the policy is general rather than targeted, if applicable. We then rescale each of
# these by their maximum value to create a score between 0 and 100, with a missing
# value contributing 0.5 These scores are then averaged to get the composite indices.

<ipython-input-7-0220f3b0c3da>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_policy[col] /= 100


In [8]:
# Only Run Once
df_policy = df_policy.reset_index().drop("index", axis=1)

In [9]:
df_policy = df_policy[df_policy['Date'] >= 20200122]


In [10]:
df_policy['Date'] = pd.to_datetime(df_policy['Date'], format='%Y%m%d')

In [11]:
states_in_policy = set(df_policy['RegionName'])

In [12]:
df_policy = df_policy.drop(['RegionCode', 'Jurisdiction'], axis=1)

In [13]:
# This cell replaces all Nan Values where the previous value == the next valid value 
# for the specified column - the Nan is replaced with the prev value
df_policy.loc[df_policy['StringencyIndexForDisplay'].ffill() == 
              df_policy['StringencyIndexForDisplay'].bfill(),
              'StringencyIndexForDisplay'] = df_policy['StringencyIndexForDisplay'].ffill()

df_policy.loc[df_policy['GovernmentResponseIndexForDisplay'].ffill() == 
              df_policy['GovernmentResponseIndexForDisplay'].bfill(),
              'GovernmentResponseIndexForDisplay'] = df_policy['GovernmentResponseIndexForDisplay'].ffill()

df_policy.loc[df_policy['ContainmentHealthIndexForDisplay'].ffill() == 
              df_policy['ContainmentHealthIndexForDisplay'].bfill(),
              'ContainmentHealthIndexForDisplay'] = df_policy['ContainmentHealthIndexForDisplay'].ffill()

df_policy.loc[df_policy['EconomicSupportIndexForDisplay'].ffill() == 
              df_policy['EconomicSupportIndexForDisplay'].bfill(),
              'EconomicSupportIndexForDisplay'] = df_policy['EconomicSupportIndexForDisplay'].ffill()

# Old code
# The following line are checking the EconomicSupportIndexForDisplay data for Idaho as
# There seems to be a NaN at 9/1 that is a one off - the surrounding numbers are zero so I will make it 0.
# y = df_policy.loc[(df_policy['RegionName']=='Idaho') & (df_policy['Date'] >= pd.Timestamp(2020,8,30))]

# The below line changes the value - for Idaho EconomicSupportIndexForDisplay on 9/1/2020 to zero
# df_policy.loc[(df_policy['RegionName']=='Idaho') &
#               (df_policy['Date'] == pd.Timestamp(2020,9,1)),'EconomicSupportIndexForDisplay'] = 0.0

In [14]:
# Checking there are no nulls in df_policy
null_policy = df_policy[df_policy.isnull().any(axis=1)]
null_policy 

Empty DataFrame
Columns: [Date, RegionName, CountryCode, StringencyIndexForDisplay, GovernmentResponseIndexForDisplay, ContainmentHealthIndexForDisplay, EconomicSupportIndexForDisplay]
Index: []

# JHU Data: Covid-19 Deaths

In [15]:
# Only Run Once
df_deaths_jhu = org_jhu_deaths_df.groupby('Province_State').sum()
df_deaths_jhu = df_deaths_jhu.reset_index()

In [16]:
# Only Run Once
df_deaths_jhu['Province_State'].replace({"District of Columbia": "Washington DC"}, inplace=True)
states_in_df_deaths = set(df_deaths_jhu['Province_State'])

# everything in policy df is in deaths df - this sees what in deaths df but not in policy
diff_states = states_in_df_deaths.difference(states_in_policy)
rows_to_remove = df_deaths_jhu[df_deaths_jhu['Province_State'].isin(diff_states)].index
df_deaths_jhu.drop(rows_to_remove, inplace=True)
df_deaths_jhu = df_deaths_jhu.reset_index()

In [17]:
state_population = df_deaths_jhu[['Province_State', 'Population']]

In [18]:
df_deaths_jhu = df_deaths_jhu.drop(['index','UID', 'code3', 'FIPS', 'Lat', 'Long_', 'Population'], axis=1)

In [19]:
date_list = list(df_deaths_jhu.columns)[1:] # index 0 has the state

In [21]:
temp_deaths_df = df_deaths_jhu.copy()
deaths_df = pd.DataFrame(columns=('Date', 'State', 'Deaths Per Capita'))

Note: This cell takes a little while to run approx 48.62 seconds

In [22]:
for index, row in temp_deaths_df.iterrows():
    for date in date_list:
        state = row['Province_State']
        new_row = pd.Series({'Date':date, 'State':state,
                             'Deaths Per Capita':row[date] / 
                             int(state_population[state_population['Province_State'] == state]['Population'])})
        deaths_df = deaths_df.append(new_row, ignore_index= True)

# JHU Data: Covid-19 Cases

In [23]:
# Only Run Once
df_cases_jhu = org_jhu_cases_df.groupby('Province_State').sum()
df_cases_jhu = df_cases_jhu.reset_index()

In [24]:
# Only Run Once
df_cases_jhu['Province_State'].replace({"District of Columbia": "Washington DC"}, inplace=True)
states_in_df_cases = set(df_cases_jhu['Province_State'])

rows_to_remove = df_cases_jhu[df_cases_jhu['Province_State'].isin(diff_states)].index
df_cases_jhu.drop(rows_to_remove, inplace=True)
df_cases_jhu = df_cases_jhu.reset_index()

In [25]:
temp_cases_df = df_cases_jhu.copy()

Note: This cell takes a little while to run approx 47.93 seconds

In [26]:
cases_df = pd.DataFrame(columns=('Date', 'State', 'Cases Per Capita'))
for index, row in temp_cases_df.iterrows():
    for date in date_list:
        state = row['Province_State']
        new_row = pd.Series({'Date':date, 'State':state,
                             'Cases Per Capita': row[date] /
                              int(state_population[state_population['Province_State'] == state]['Population'])})
        cases_df = cases_df.append(new_row, ignore_index= True)

In [27]:
# cases_df

In [28]:
# deaths_df

In [29]:
# df_policy

In [30]:
deaths_and_cases_df = pd.merge(deaths_df, cases_df,  how='left', left_on=['Date','State'], 
                            right_on = ['Date','State'])
deaths_and_cases_df['Date'] = pd.to_datetime(deaths_and_cases_df['Date'], format='%m/%d/%y')

In [31]:
# deaths_and_cases_df ends at 10/26 while df_policy ends at 10/27
# Here we are setting restrictingt the dates of df_policy to be stop at the max date of deaths_and_cases_df
df_policy = df_policy[df_policy['Date'] <=  deaths_and_cases_df['Date'].max()]

In [32]:
len(df_policy) == len(deaths_and_cases_df)

True

In [33]:
covid_combined_df = pd.merge(deaths_and_cases_df, df_policy, how='inner',
                               left_on=['Date', 'State'], right_on=['Date', 'RegionName'])

In [34]:
# Checking there is no null values in the new  combined df
temp = covid_combined_df[covid_combined_df.isnull().any(axis=1)]
temp

Empty DataFrame
Columns: [Date, State, Deaths Per Capita, Cases Per Capita, RegionName, CountryCode, StringencyIndexForDisplay, GovernmentResponseIndexForDisplay, ContainmentHealthIndexForDisplay, EconomicSupportIndexForDisplay]
Index: []

In [35]:
# Checking to confirm the number of rows in our dataframes are consistent and the new df is the correct len
len(covid_combined_df) == len(df_policy) == len(deaths_and_cases_df)

True

In [36]:
covid_combined_df = covid_combined_df.drop(['RegionName', 'CountryCode'], axis=1)

In [37]:
cols = list(covid_combined_df.columns.values)
cols.remove('Cases Per Capita')
cols.remove('Deaths Per Capita')
cols.append('Cases Per Capita')
cols.append('Deaths Per Capita')
# cols

In [38]:
covid_combined_df = covid_combined_df[cols]

In [39]:
def create_location_dict(df, location_col_name):
    location_dict ={}
    location_num = 0
    for locale in df[location_col_name].unique():
        location_dict[locale] = location_num
        location_num += 1
    return location_dict

state_dict = create_location_dict(covid_combined_df, 'State')
# state_dict

In [40]:
df = covid_combined_df.copy()
df["State"].replace(state_dict, inplace=True)

In [41]:
# Shrink df to start at relevant start date
# chose 2/28/2020 b/c the first date will get removed when turning it into timeseries
# So you will be left with 3/1/2020
df = df[df['Date'] >= "02/29/2020"]

In [42]:
df_deaths = df.drop(columns='Cases Per Capita')
df_cases = df.drop(columns='Deaths Per Capita')

In [43]:
# df_deaths

In [44]:
# df_cases

In [45]:
# This code is from:
#     https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/

# from pandas import DataFrame - nw note: adj calls using pd.___ 
# from pandas import concat - nw note: adj calls using pd.___
 
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg



In [46]:
# Create new dataframe's for time series data
ts_death_data = pd.DataFrame()
ts_cases_data = pd.DataFrame()

# Look at df one state at a time.
def prep_ts(df, location_col_name, new_df):
    for locale in df[location_col_name].unique():
        locale_df = df[df[location_col_name] == locale]
        locale_df = locale_df.to_numpy()
        locale_df = series_to_supervised(locale_df)
        new_df = pd.concat([new_df, locale_df])
    return new_df

ts_death_df = prep_ts(df_deaths, 'State', ts_death_data)
ts_cases_df = prep_ts(df_cases, 'State', ts_cases_data)   

In [47]:
# renameing the columns in the time series dataframes
def rename_cols_dict(before_df, ts_df):
    detailed_cols_list = []
    org_cols = before_df.columns
    for i in ["(t-1)","(t)"]:
        for col in org_cols:
            detailed_cols_list.append(col+i)
    update_cols_dict = dict(zip(ts_df.columns, detailed_cols_list))
    return update_cols_dict

In [48]:
# Only run once
ts_death_df.rename(rename_cols_dict(df_deaths, ts_death_df), axis=1, inplace=True)
ts_death_df.reset_index(inplace=True)
ts_cases_df.rename(rename_cols_dict(df_deaths, ts_cases_df), axis=1, inplace=True)
ts_cases_df.reset_index(inplace=True)

In [49]:
# Next i need to split data into train and test - NEED TO DO FOR CASES

# Deaths - model input data, tuning, and creation.

In [50]:
# This is for ts_death_df

# last 5 days of each month to be used for test
march_test_rows = list(ts_death_df.loc[(ts_death_df['Date(t)'] >= "03/27/2020") 
                                       & (ts_death_df['Date(t)'] <= "03/31/2020")].index)
april_test_rows = list(ts_death_df.loc[(ts_death_df['Date(t)'] >= "04/26/2020") 
                                       & (ts_death_df['Date(t)'] <= "04/30/2020")].index)
may_test_rows = list(ts_death_df.loc[(ts_death_df['Date(t)'] >= "05/27/2020") 
                                       & (ts_death_df['Date(t)'] <= "05/31/2020")].index)
june_test_rows = list(ts_death_df.loc[(ts_death_df['Date(t)'] >= "06/26/2020") 
                                       & (ts_death_df['Date(t)'] <= "06/30/2020")].index)
july_test_rows = list(list(ts_death_df.loc[(ts_death_df['Date(t)'] >= "07/27/2020") 
                                       & (ts_death_df['Date(t)'] <= "07/31/2020")].index))
august_test_rows = list(ts_death_df.loc[(ts_death_df['Date(t)'] >= "08/27/2020") 
                                       & (ts_death_df['Date(t)'] <= "08/31/2020")].index)
sept_test_rows = list(ts_death_df.loc[(ts_death_df['Date(t)'] >= "09/26/2020") 
                                       & (ts_death_df['Date(t)'] <= "09/30/2020")].index)
oct_test_rows = list(ts_death_df.loc[(ts_death_df['Date(t)'] >= "10/22/2020") 
                                       & (ts_death_df['Date(t)'] <= "10/26/2020")].index)
test_indices = march_test_rows + april_test_rows + may_test_rows + june_test_rows \
                + july_test_rows + august_test_rows + sept_test_rows + oct_test_rows
# test_indices


## If I only use one day per month to train
# march_test_rows=list(ts_death_df[ts_death_df["Date(t)"]== "03/31/2020"].index)
# april_test_rows=list(ts_death_df[ts_death_df["Date(t)"]== "04/30/2020"].index)
# may_test_rows=list(ts_death_df[ts_death_df["Date(t)"]== "05/31/2020"].index)
# june_test_rows=list(ts_death_df[ts_death_df["Date(t)"]== "06/30/20200"].index)
# july_test_rows=list(ts_death_df[ts_death_df["Date(t)"]== "07/31/2020"].index)
# august_test_rows=list(ts_death_df[ts_death_df["Date(t)"]== "08/31/20200"].index)
# sept_test_rows=list(ts_death_df[ts_death_df["Date(t)"]== "09/30/20200"].index)
# oct_test_rows=list(ts_death_df[ts_death_df["Date(t)"]== "10/26/20200"].index)
# test_indices = march_test_rows + april_test_rows + may_test_rows + june_test_rows \
#                 + july_test_rows + august_test_rows + sept_test_rows + oct_test_rows


In [51]:
death_train = ts_death_df[~ts_death_df.index.isin(test_indices)]
death_test = ts_death_df[ts_death_df.index.isin(test_indices)]

In [52]:
# Need to remove the dates because they will cause the below error
# TypeError: float() argument must be a string or a number, not 'Timestamp'
death_train = death_train.drop(['index', "Date(t-1)", "Date(t)"], axis=1)
death_test = death_test.drop(['index', "Date(t-1)", "Date(t)"], axis=1)

In [53]:
# the label is the last column which is the last index of the array
# this splits the data into x and y 
death_train_x = death_train.values[: , :-1] 
death_train_y = death_train.values[: , -1]
death_test_x = death_test.values[: , :-1] 
death_test_y = death_test.values[: , -1]

In [54]:
# The data needs to be reshaped so it is 3d for the lstm
death_train_x = death_train_x.reshape((death_train_x.shape[0], 1, death_train_x.shape[1]))
death_test_x = death_test_x.reshape((death_test_x.shape[0], 1, death_test_x.shape[1]))

# The data needs to be type float32 for this to train (similar to why we removed the dates column)
death_train_x = np.asarray(death_train_x).astype(np.float32)
death_train_y = np.asarray(death_train_y).astype(np.float32)
death_test_x = np.asarray(death_test_x).astype(np.float32)
death_test_y = np.asarray(death_test_y).astype(np.float32)

Hyperparameter Tuning - Covid Deaths Model

In [124]:
# We have the results of the tuning so this does not need to be trained again. It takes over an hour

# start = time.time()

# outcomes = []
# for batch_size in range(50, 200, 50):
#     for epochs in range(15,30,5):
#         for lstm_num in range(15,30,5):
#             for dense1 in range(15,30,5):
#                 for dense2 in range(15,30,5):
#                     for dense3 in range(15,30,5):
#                         outcome = [batch_size, epochs, lstm_num, dense1, dense2, dense3]
#                         model = Sequential()
#                         model.add(LSTM(lstm_num, input_shape = (death_train_x.shape[1], death_train_x.shape[2])))
#                         model.add(Dense(dense1, activation="relu"))
#                         model.add(Dense(dense2, activation="relu"))
#                         model.add(Dense(dense3, activation="relu"))
#                         model.add(Dense(1))

#                         model.compile(optimizer='adam', loss='mse',)
#                         fit=model.fit(
#                               death_train_x,
#                               death_train_y,
#                               epochs=epochs,
#                               validation_data=(death_test_x, death_test_y),
#                               batch_size=batch_size,)
#                         result = model.evaluate(death_test_x, death_test_y)
#                         outcome.append(result)
#                         outcomes.append(outcome)

Epoch 1/15
208/208 [==============================] - 0s 2ms/step - loss: 1.3569e-04 - val_loss: 1.1754e-06
Epoch 2/15
208/208 [==============================] - 0s 1ms/step - loss: 7.0147e-07 - val_loss: 3.4785e-07
Epoch 3/15
208/208 [==============================] - 0s 1ms/step - loss: 3.1242e-07 - val_loss: 2.2090e-07
Epoch 4/15
208/208 [==============================] - 0s 1ms/step - loss: 2.1444e-07 - val_loss: 1.7056e-07
Epoch 5/15
208/208 [==============================] - 0s 1ms/step - loss: 1.7875e-07 - val_loss: 1.5081e-07
Epoch 6/15
208/208 [==============================] - 0s 2ms/step - loss: 1.6201e-07 - val_loss: 1.7930e-07
Epoch 7/15
208/208 [==============================] - 0s 1ms/step - loss: 1.4118e-07 - val_loss: 1.2849e-07
Epoch 8/15
208/208 [==============================] - 0s 1ms/step - loss: 1.2973e-07 - val_loss: 1.1712e-07
Epoch 9/15
208/208 [==============================] - 0s 1ms/step - loss: 1.2924e-07 - val_loss: 1.2965e-07
Epoch 10/15
208/208 [=======

Epoch 14/15
208/208 [==============================] - 0s 1ms/step - loss: 1.2002e-07 - val_loss: 1.1273e-07
Epoch 15/15
65/65 [==============================] - 0s 715us/step - loss: 1.7983e-07
Epoch 1/15
208/208 [==============================] - 1s 2ms/step - loss: 4.5154e-05 - val_loss: 7.6515e-07
Epoch 2/15
208/208 [==============================] - 0s 1ms/step - loss: 4.6874e-07 - val_loss: 3.7097e-07
Epoch 3/15
208/208 [==============================] - 0s 1ms/step - loss: 2.7960e-07 - val_loss: 2.3413e-07
Epoch 4/15
208/208 [==============================] - 0s 1ms/step - loss: 2.1175e-07 - val_loss: 1.9580e-07
Epoch 5/15
208/208 [==============================] - 0s 1ms/step - loss: 1.8013e-07 - val_loss: 1.7322e-07
Epoch 6/15
208/208 [==============================] - 0s 1ms/step - loss: 1.6801e-07 - val_loss: 1.7180e-07
Epoch 7/15
208/208 [==============================] - 0s 1ms/step - loss: 1.5675e-07 - val_loss: 1.6154e-07
Epoch 8/15
208/208 [=============================

Epoch 11/15
208/208 [==============================] - 0s 1ms/step - loss: 2.0560e-07 - val_loss: 1.0572e-07
Epoch 12/15
208/208 [==============================] - 0s 1ms/step - loss: 2.2454e-07 - val_loss: 1.6539e-07
Epoch 13/15
208/208 [==============================] - 0s 1ms/step - loss: 2.5444e-07 - val_loss: 3.9778e-07
Epoch 14/15
208/208 [==============================] - 0s 1ms/step - loss: 3.8188e-07 - val_loss: 2.0412e-07
Epoch 15/15
65/65 [==============================] - 0s 717us/step - loss: 2.0584e-07
Epoch 1/15
208/208 [==============================] - 0s 2ms/step - loss: 5.0574e-05 - val_loss: 4.3899e-07
Epoch 2/15
208/208 [==============================] - 0s 1ms/step - loss: 4.1529e-07 - val_loss: 3.0533e-07
Epoch 3/15
208/208 [==============================] - 0s 1ms/step - loss: 3.2115e-07 - val_loss: 2.7671e-07
Epoch 4/15
208/208 [==============================] - 0s 1ms/step - loss: 2.7781e-07 - val_loss: 2.9955e-07
Epoch 5/15
208/208 [==========================

208/208 [==============================] - 0s 1ms/step - loss: 1.6046e-07 - val_loss: 1.9136e-07
Epoch 9/15
208/208 [==============================] - 0s 1ms/step - loss: 1.4403e-07 - val_loss: 1.2268e-07
Epoch 10/15
208/208 [==============================] - 0s 1ms/step - loss: 1.4079e-07 - val_loss: 1.8830e-07
Epoch 11/15
208/208 [==============================] - 0s 1ms/step - loss: 1.3440e-07 - val_loss: 1.3194e-07
Epoch 12/15
208/208 [==============================] - 0s 1ms/step - loss: 1.2146e-07 - val_loss: 1.3866e-07
Epoch 13/15
208/208 [==============================] - 0s 1ms/step - loss: 1.0948e-07 - val_loss: 9.1327e-08
Epoch 14/15
208/208 [==============================] - 0s 1ms/step - loss: 1.1621e-07 - val_loss: 9.3460e-08
Epoch 15/15
65/65 [==============================] - 0s 701us/step - loss: 1.1776e-07
Epoch 1/15
208/208 [==============================] - 0s 2ms/step - loss: 1.8128e-04 - val_loss: 7.1559e-07
Epoch 2/15
208/208 [==============================] - 0s

208/208 [==============================] - 0s 1ms/step - loss: 1.6748e-07 - val_loss: 1.4069e-07
Epoch 6/15
208/208 [==============================] - 0s 1ms/step - loss: 1.6150e-07 - val_loss: 1.3770e-07
Epoch 7/15
208/208 [==============================] - 0s 1ms/step - loss: 1.9261e-07 - val_loss: 2.5706e-07
Epoch 8/15
208/208 [==============================] - 0s 1ms/step - loss: 1.7704e-07 - val_loss: 1.9694e-07
Epoch 9/15
208/208 [==============================] - 0s 1ms/step - loss: 1.4319e-07 - val_loss: 1.5840e-07
Epoch 10/15
208/208 [==============================] - 0s 1ms/step - loss: 1.5431e-07 - val_loss: 9.5621e-08
Epoch 11/15
208/208 [==============================] - 0s 1ms/step - loss: 1.2584e-07 - val_loss: 1.0234e-07
Epoch 12/15
208/208 [==============================] - 0s 1ms/step - loss: 1.5403e-07 - val_loss: 1.5234e-07
Epoch 13/15
208/208 [==============================] - 0s 1ms/step - loss: 1.6612e-07 - val_loss: 1.6490e-07
Epoch 14/15
208/208 [==============

...

Epoch 2/25
70/70 [==============================] - 0s 2ms/step - loss: 1.1562e-06 - val_loss: 5.8642e-07
Epoch 3/25
70/70 [==============================] - 0s 2ms/step - loss: 5.5486e-07 - val_loss: 3.8327e-07
Epoch 4/25
70/70 [==============================] - 0s 2ms/step - loss: 3.7495e-07 - val_loss: 2.9725e-07
Epoch 5/25
70/70 [==============================] - 0s 2ms/step - loss: 2.9535e-07 - val_loss: 2.4979e-07
Epoch 6/25
70/70 [==============================] - 0s 2ms/step - loss: 2.5250e-07 - val_loss: 2.2169e-07
Epoch 7/25
70/70 [==============================] - 0s 2ms/step - loss: 2.2652e-07 - val_loss: 2.0202e-07
Epoch 8/25
70/70 [==============================] - 0s 2ms/step - loss: 2.0666e-07 - val_loss: 1.8794e-07
Epoch 9/25
70/70 [==============================] - 0s 2ms/step - loss: 1.8865e-07 - val_loss: 1.8981e-07
Epoch 10/25
70/70 [==============================] - 0s 2ms/step - loss: 1.7792e-07 - val_loss: 1.6316e-07
Epoch 11/25
70/70 [==========================

Epoch 2/25
70/70 [==============================] - 0s 2ms/step - loss: 2.7868e-06 - val_loss: 1.9564e-06
Epoch 3/25
70/70 [==============================] - 0s 2ms/step - loss: 1.6167e-06 - val_loss: 1.2691e-06
Epoch 4/25
70/70 [==============================] - 0s 2ms/step - loss: 1.1768e-06 - val_loss: 9.6386e-07
Epoch 5/25
70/70 [==============================] - 0s 2ms/step - loss: 9.5457e-07 - val_loss: 8.2275e-07
Epoch 6/25
70/70 [==============================] - 0s 2ms/step - loss: 8.2908e-07 - val_loss: 1.0527e-06
Epoch 7/25
70/70 [==============================] - 0s 2ms/step - loss: 7.5194e-07 - val_loss: 6.7148e-07
Epoch 8/25
70/70 [==============================] - 0s 2ms/step - loss: 6.8842e-07 - val_loss: 6.2493e-07
Epoch 9/25
70/70 [==============================] - 0s 2ms/step - loss: 6.2400e-07 - val_loss: 5.6616e-07
Epoch 10/25
70/70 [==============================] - 0s 2ms/step - loss: 5.8443e-07 - val_loss: 6.2942e-07
Epoch 11/25
70/70 [==========================

Epoch 2/25
70/70 [==============================] - 0s 2ms/step - loss: 7.9300e-06 - val_loss: 6.0477e-06
Epoch 3/25
70/70 [==============================] - 0s 2ms/step - loss: 4.5067e-06 - val_loss: 3.3172e-06
Epoch 4/25
70/70 [==============================] - 0s 2ms/step - loss: 2.2332e-06 - val_loss: 1.1531e-06
Epoch 5/25
70/70 [==============================] - 0s 2ms/step - loss: 9.4093e-07 - val_loss: 6.7905e-07
Epoch 6/25
70/70 [==============================] - 0s 2ms/step - loss: 6.4200e-07 - val_loss: 5.2125e-07
Epoch 7/25
70/70 [==============================] - 0s 2ms/step - loss: 5.0174e-07 - val_loss: 4.2372e-07
Epoch 8/25
70/70 [==============================] - 0s 2ms/step - loss: 4.1940e-07 - val_loss: 3.6160e-07
Epoch 9/25
70/70 [==============================] - 0s 2ms/step - loss: 3.6750e-07 - val_loss: 3.2472e-07
Epoch 10/25
70/70 [==============================] - 0s 2ms/step - loss: 3.3539e-07 - val_loss: 3.2585e-07
Epoch 11/25
70/70 [==========================

Epoch 2/25
70/70 [==============================] - 0s 2ms/step - loss: 1.7683e-06 - val_loss: 1.1437e-06
Epoch 3/25
70/70 [==============================] - 0s 2ms/step - loss: 1.1118e-06 - val_loss: 9.2299e-07
Epoch 4/25
70/70 [==============================] - 0s 2ms/step - loss: 8.9534e-07 - val_loss: 7.1284e-07
Epoch 5/25
70/70 [==============================] - 0s 2ms/step - loss: 7.3498e-07 - val_loss: 6.0175e-07
Epoch 6/25
70/70 [==============================] - 0s 2ms/step - loss: 6.2225e-07 - val_loss: 5.2740e-07
Epoch 7/25
70/70 [==============================] - 0s 2ms/step - loss: 5.2321e-07 - val_loss: 4.9500e-07
Epoch 8/25
70/70 [==============================] - 0s 2ms/step - loss: 4.6927e-07 - val_loss: 4.2794e-07
Epoch 9/25
70/70 [==============================] - 0s 2ms/step - loss: 4.1615e-07 - val_loss: 3.9807e-07
Epoch 10/25
70/70 [==============================] - 0s 2ms/step - loss: 3.8236e-07 - val_loss: 3.4173e-07
Epoch 11/25
70/70 [==========================

Epoch 2/25
70/70 [==============================] - 0s 2ms/step - loss: 2.1302e-06 - val_loss: 1.3392e-06
Epoch 3/25
70/70 [==============================] - 0s 2ms/step - loss: 1.2263e-06 - val_loss: 7.4948e-07
Epoch 4/25
70/70 [==============================] - 0s 2ms/step - loss: 7.1543e-07 - val_loss: 5.0374e-07
Epoch 5/25
70/70 [==============================] - 0s 2ms/step - loss: 5.1519e-07 - val_loss: 4.1966e-07
Epoch 6/25
70/70 [==============================] - 0s 2ms/step - loss: 4.1806e-07 - val_loss: 3.8251e-07
Epoch 7/25
70/70 [==============================] - 0s 2ms/step - loss: 3.7418e-07 - val_loss: 3.5195e-07
Epoch 8/25
70/70 [==============================] - 0s 2ms/step - loss: 3.4970e-07 - val_loss: 3.3484e-07
Epoch 9/25
70/70 [==============================] - 0s 2ms/step - loss: 3.2569e-07 - val_loss: 3.3903e-07
Epoch 10/25
70/70 [==============================] - 0s 2ms/step - loss: 3.1085e-07 - val_loss: 2.9611e-07
Epoch 11/25
70/70 [==========================

In [55]:
# stop = time.time() 

In [126]:
# train_time = stop - start
# train_time
# # train_time = 3581.909922838211
# # This took about an hour to train all the permuations of the hyperparameters

4286.875309467316

In [127]:
# # this sorts the outcomes list of lists by the last index of the sub list
# # which is the loss. 
# outcomes.sort(key=lambda x: x[-1])
# outcomes

[[50, 25, 25, 20, 25, 20, 6.103122984768561e-08],
 [50, 25, 25, 20, 20, 15, 6.499617910549205e-08],
 [50, 25, 20, 25, 25, 20, 6.560831877777673e-08],
 [50, 25, 15, 20, 20, 20, 6.585398892866579e-08],
 [150, 20, 25, 25, 20, 15, 7.193761319967962e-08],
 [50, 25, 20, 25, 25, 25, 7.306690719133258e-08],
 [50, 15, 25, 15, 25, 25, 7.478409003169872e-08],
 [100, 25, 20, 15, 25, 15, 7.648423405726135e-08],
 [50, 15, 20, 20, 20, 15, 7.661697054572869e-08],
 [50, 25, 15, 25, 15, 15, 7.735918927664898e-08],
 [50, 20, 20, 15, 25, 20, 7.746789520979291e-08],
 [50, 25, 15, 25, 15, 25, 7.893693521054956e-08],
 [50, 25, 25, 25, 15, 20, 7.936159818200395e-08],
 [50, 25, 20, 20, 20, 15, 7.952733227511999e-08],
 [50, 15, 20, 25, 20, 20, 8.042319876722104e-08],
 [100, 25, 25, 20, 25, 20, 8.06233018124658e-08],
 [50, 25, 15, 25, 25, 25, 8.065666179390973e-08],
 [50, 25, 15, 15, 25, 25, 8.148768415594532e-08],
 [50, 25, 15, 15, 15, 15, 8.326143330350533e-08],
 [50, 25, 25, 15, 20, 15, 8.574913579195709e-08]

In [59]:
from statistics import mode
# Here we take the first 5 items in the sorted outcomes which means the 10 combos with the lowest loss. 

# outcome_check = outcomes[:5] # commenting this out and replacing with results
# This is the first 5 indices from the last run of the above hyperparameter tuning
outcome_check = [[50, 25, 25, 20, 25, 20, 6.103122984768561e-08],
                 [50, 25, 25, 20, 20, 15, 6.499617910549205e-08],
                 [50, 25, 20, 25, 25, 20, 6.560831877777673e-08],
                 [50, 25, 15, 20, 20, 20, 6.585398892866579e-08],
                 [150, 20, 25, 25, 20, 15, 7.193761319967962e-08]]

# Now we break up the items into seperate lists
batch_lst, epoch_lst, lstm_num_lst, dense1_lst, dense2_lst, dense3_lst = [],[],[],[],[],[]
for lst in outcome_check:
    batch_lst.append(lst[0])
    epoch_lst.append(lst[1])
    lstm_num_lst.append(lst[2])
    dense1_lst.append(lst[3])
    dense2_lst.append(lst[4])
    dense3_lst.append(lst[5])
# Now we create a dictionary which will take the mode of each hyperparameter and put it in the dictionary
tuning_dict = {}
tuning_dict['batch_size'] = mode(batch_lst)
tuning_dict['epochs'] = mode(epoch_lst)
tuning_dict['lst_num'] = mode(lstm_num_lst)
tuning_dict['dense1'] = mode(dense1_lst)
tuning_dict['dense2'] = mode(dense2_lst)
tuning_dict['dense3'] = mode(dense3_lst)

tuning_dict
# # this is the result: 
# {'batch_size': 50,
#  'epochs': 25,
#  'lst_num': 25,
#  'dense1': 20,
#  'dense2': 20,
#  'dense3': 20}

{'batch_size': 50,
 'epochs': 25,
 'lst_num': 25,
 'dense1': 20,
 'dense2': 20,
 'dense3': 20}

In [64]:
batch_size = tuning_dict['batch_size']
epochs = tuning_dict['epochs']
lstm_num = tuning_dict['lst_num']
dense1 = tuning_dict['dense1']
dense2 = tuning_dict['dense2']
dense3 = tuning_dict['dense3']


model = Sequential()
model.add(LSTM(lstm_num, input_shape = (death_train_x.shape[1], death_train_x.shape[2])))
model.add(Dense(dense1, activation="relu"))
model.add(Dense(dense2, activation="relu"))
model.add(Dense(dense3, activation="relu"))
model.add(Dense(1))
                    
model.compile(optimizer='adam', loss='mse',)

fit=model.fit(
      death_train_x,
      death_train_y,
      epochs=epochs,
      validation_data=(death_test_x, death_test_y),
      batch_size=batch_size,)

model.evaluate(death_test_x, death_test_y)

Epoch 1/25
208/208 [==============================] - 0s 2ms/step - loss: 4.1936e-06 - val_loss: 1.8773e-07
Epoch 2/25
208/208 [==============================] - 0s 1ms/step - loss: 1.6973e-07 - val_loss: 1.1192e-07
Epoch 3/25
208/208 [==============================] - 0s 1ms/step - loss: 1.0917e-07 - val_loss: 1.0395e-07
Epoch 4/25
208/208 [==============================] - 0s 1ms/step - loss: 9.4747e-08 - val_loss: 9.8043e-08
Epoch 5/25
208/208 [==============================] - 0s 1ms/step - loss: 9.7063e-08 - val_loss: 1.4334e-07
Epoch 6/25
208/208 [==============================] - 0s 1ms/step - loss: 1.1239e-07 - val_loss: 2.2546e-07
Epoch 7/25
208/208 [==============================] - 0s 2ms/step - loss: 9.3496e-08 - val_loss: 9.3652e-08
Epoch 8/25
208/208 [==============================] - 0s 1ms/step - loss: 1.3379e-07 - val_loss: 1.7621e-07
Epoch 9/25
208/208 [==============================] - 0s 1ms/step - loss: 1.2086e-07 - val_loss: 7.9965e-08
Epoch 10/25
208/208 [=======

6.806989460983459e-08

In [65]:
model.save("covid_deaths_model.h5")

# Cases - model input data, tuning, and creation.

In [66]:
# This is for ts_cases_df

# last 5 days of each month to be used for test
march_test_rows = list(ts_cases_df.loc[(ts_cases_df['Date(t)'] >= "03/27/2020") 
                                       & (ts_cases_df['Date(t)'] <= "03/31/2020")].index)
april_test_rows = list(ts_cases_df.loc[(ts_cases_df['Date(t)'] >= "04/26/2020") 
                                       & (ts_cases_df['Date(t)'] <= "04/30/2020")].index)
may_test_rows = list(ts_cases_df.loc[(ts_cases_df['Date(t)'] >= "05/27/2020") 
                                       & (ts_cases_df['Date(t)'] <= "05/31/2020")].index)
june_test_rows = list(ts_cases_df.loc[(ts_cases_df['Date(t)'] >= "06/26/2020") 
                                       & (ts_cases_df['Date(t)'] <= "06/30/2020")].index)
july_test_rows = list(list(ts_cases_df.loc[(ts_cases_df['Date(t)'] >= "07/27/2020") 
                                       & (ts_cases_df['Date(t)'] <= "07/31/2020")].index))
august_test_rows = list(ts_cases_df.loc[(ts_cases_df['Date(t)'] >= "08/27/2020") 
                                       & (ts_cases_df['Date(t)'] <= "08/31/2020")].index)
sept_test_rows = list(ts_cases_df.loc[(ts_cases_df['Date(t)'] >= "09/26/2020") 
                                       & (ts_cases_df['Date(t)'] <= "09/30/2020")].index)
oct_test_rows = list(ts_cases_df.loc[(ts_cases_df['Date(t)'] >= "10/22/2020") 
                                       & (ts_cases_df['Date(t)'] <= "10/26/2020")].index)
test_indices_cases = march_test_rows + april_test_rows + may_test_rows + june_test_rows \
                + july_test_rows + august_test_rows + sept_test_rows + oct_test_rows
# test_indices


In [67]:
cases_train = ts_cases_df[~ts_cases_df.index.isin(test_indices)]
cases_test = ts_cases_df[ts_cases_df.index.isin(test_indices)]

# Need to remove the dates because they will cause the below error
# TypeError: float() argument must be a string or a number, not 'Timestamp'
cases_train = cases_train.drop(['index', "Date(t-1)", "Date(t)"], axis=1)
cases_test = cases_test.drop(['index', "Date(t-1)", "Date(t)"], axis=1)

In [68]:
# the label is the last column which is the last index of the array
# this splits the data into x and y 
cases_train_x = cases_train.values[: , :-1] 
cases_train_y = cases_train.values[: , -1]
cases_test_x = cases_test.values[: , :-1] 
cases_test_y = cases_test.values[: , -1]

In [69]:
cases_train_x = cases_train_x.reshape((cases_train_x.shape[0], 1, cases_train_x.shape[1]))
cases_test_x = cases_test_x.reshape((cases_test_x.shape[0], 1, cases_test_x.shape[1]))

cases_train_x = np.asarray(cases_train_x).astype(np.float32)
cases_train_y = np.asarray(cases_train_y).astype(np.float32)
cases_test_x = np.asarray(cases_test_x).astype(np.float32)
cases_test_y = np.asarray(cases_test_y).astype(np.float32)

In [73]:
start = time.time()

outcomes_cases = []
for batch_size in range(50, 200, 50):
    for epochs in range(15,30,5):
        for lstm_num in range(15,30,5):
            for dense1 in range(15,30,5):
                for dense2 in range(15,30,5):
                    for dense3 in range(15,30,5):
                        outcome = [batch_size, epochs, lstm_num, dense1, dense2, dense3]
                        model = Sequential()
                        model.add(LSTM(lstm_num, input_shape = (cases_train_x.shape[1], cases_train_x.shape[2])))
                        model.add(Dense(dense1, activation="relu"))
                        model.add(Dense(dense2, activation="relu"))
                        model.add(Dense(dense3, activation="relu"))
                        model.add(Dense(1))

                        model.compile(optimizer='adam', loss='mse',)
                        fit=model.fit(
                              cases_train_x,
                              cases_train_y,
                              epochs=epochs,
                              validation_data=(cases_test_x, cases_test_y),
                              batch_size=batch_size,)
                        result = model.evaluate(cases_test_x, cases_test_y)
                        outcome.append(result)
                        outcomes_cases.append(outcome)

Epoch 1/15
208/208 [==============================] - 0s 2ms/step - loss: 3.4090e-04 - val_loss: 9.3484e-05
Epoch 2/15
208/208 [==============================] - 0s 1ms/step - loss: 6.6912e-05 - val_loss: 8.9776e-05
Epoch 3/15
208/208 [==============================] - 0s 1ms/step - loss: 6.2871e-05 - val_loss: 8.3291e-05
Epoch 4/15
208/208 [==============================] - 0s 1ms/step - loss: 5.8197e-05 - val_loss: 8.1685e-05
Epoch 5/15
208/208 [==============================] - 0s 1ms/step - loss: 5.2844e-05 - val_loss: 6.8711e-05
Epoch 6/15
208/208 [==============================] - 0s 1ms/step - loss: 4.7600e-05 - val_loss: 5.6553e-05
Epoch 7/15
208/208 [==============================] - 0s 1ms/step - loss: 4.1267e-05 - val_loss: 4.9493e-05
Epoch 8/15
208/208 [==============================] - 0s 1ms/step - loss: 3.6695e-05 - val_loss: 5.7133e-05
Epoch 9/15
208/208 [==============================] - 0s 1ms/step - loss: 3.1250e-05 - val_loss: 4.6885e-05
Epoch 10/15
208/208 [=======

Epoch 14/15
208/208 [==============================] - 0s 1ms/step - loss: 1.0687e-05 - val_loss: 1.7191e-05
Epoch 15/15
65/65 [==============================] - 0s 694us/step - loss: 6.5871e-06
Epoch 1/15
208/208 [==============================] - 0s 2ms/step - loss: 1.3107e-04 - val_loss: 1.0208e-04
Epoch 2/15
208/208 [==============================] - 0s 1ms/step - loss: 7.4421e-05 - val_loss: 9.9315e-05
Epoch 3/15
208/208 [==============================] - 0s 1ms/step - loss: 7.4461e-05 - val_loss: 9.2233e-05
Epoch 4/15
208/208 [==============================] - 0s 1ms/step - loss: 6.9906e-05 - val_loss: 8.5227e-05
Epoch 5/15
208/208 [==============================] - 0s 1ms/step - loss: 6.3742e-05 - val_loss: 7.6447e-05
Epoch 6/15
208/208 [==============================] - 0s 1ms/step - loss: 5.6616e-05 - val_loss: 8.7435e-05
Epoch 7/15
208/208 [==============================] - 0s 1ms/step - loss: 4.6512e-05 - val_loss: 5.4874e-05
Epoch 8/15
208/208 [=============================

208/208 [==============================] - 0s 1ms/step - loss: 1.5646e-05 - val_loss: 1.8959e-05
Epoch 12/15
208/208 [==============================] - 0s 1ms/step - loss: 1.4672e-05 - val_loss: 1.3749e-05
Epoch 13/15
208/208 [==============================] - 0s 1ms/step - loss: 1.2246e-05 - val_loss: 1.2525e-05
Epoch 14/15
208/208 [==============================] - 0s 1ms/step - loss: 9.6995e-06 - val_loss: 1.4158e-05
Epoch 15/15
65/65 [==============================] - 0s 712us/step - loss: 8.3639e-06
Epoch 1/15
208/208 [==============================] - 0s 2ms/step - loss: 9.5982e-04 - val_loss: 1.0159e-04
Epoch 2/15
208/208 [==============================] - 0s 1ms/step - loss: 7.3238e-05 - val_loss: 9.8139e-05
Epoch 3/15
208/208 [==============================] - 0s 1ms/step - loss: 7.0807e-05 - val_loss: 9.5028e-05
Epoch 4/15
208/208 [==============================] - 0s 1ms/step - loss: 6.6335e-05 - val_loss: 8.7614e-05
Epoch 5/15
208/208 [==============================] - 0s 1

208/208 [==============================] - 1s 2ms/step - loss: 3.0753e-05 - val_loss: 6.2010e-05
Epoch 9/15
208/208 [==============================] - 0s 1ms/step - loss: 2.6210e-05 - val_loss: 3.1231e-05
Epoch 10/15
208/208 [==============================] - 0s 1ms/step - loss: 2.4577e-05 - val_loss: 2.5258e-05
Epoch 11/15
208/208 [==============================] - 0s 1ms/step - loss: 2.4041e-05 - val_loss: 2.3527e-05
Epoch 12/15
208/208 [==============================] - 0s 1ms/step - loss: 1.7868e-05 - val_loss: 2.0775e-05
Epoch 13/15
208/208 [==============================] - 0s 1ms/step - loss: 1.6102e-05 - val_loss: 2.1099e-05
Epoch 14/15
208/208 [==============================] - 0s 1ms/step - loss: 1.4448e-05 - val_loss: 1.8389e-05
Epoch 15/15
65/65 [==============================] - 0s 721us/step - loss: 1.1888e-05
Epoch 1/15
208/208 [==============================] - 0s 2ms/step - loss: 1.7842e-04 - val_loss: 9.4274e-05
Epoch 2/15
208/208 [==============================] - 0s

208/208 [==============================] - 0s 1ms/step - loss: 4.4408e-05 - val_loss: 4.6198e-05
Epoch 6/15
208/208 [==============================] - 0s 1ms/step - loss: 3.4898e-05 - val_loss: 3.8799e-05
Epoch 7/15
208/208 [==============================] - 0s 1ms/step - loss: 2.8709e-05 - val_loss: 3.4485e-05
Epoch 8/15
208/208 [==============================] - 0s 1ms/step - loss: 2.5920e-05 - val_loss: 3.3425e-05
Epoch 9/15
208/208 [==============================] - 0s 1ms/step - loss: 2.4802e-05 - val_loss: 3.3074e-05
Epoch 10/15
208/208 [==============================] - 0s 1ms/step - loss: 1.9569e-05 - val_loss: 2.8555e-05
Epoch 11/15
208/208 [==============================] - 0s 1ms/step - loss: 1.8514e-05 - val_loss: 2.3493e-05
Epoch 12/15
208/208 [==============================] - 0s 1ms/step - loss: 1.8177e-05 - val_loss: 2.1938e-05
Epoch 13/15
208/208 [==============================] - 0s 1ms/step - loss: 1.3847e-05 - val_loss: 2.3475e-05
Epoch 14/15
208/208 [==============

...

70/70 [==============================] - 0s 2ms/step - loss: 7.6160e-05 - val_loss: 9.8937e-05
Epoch 5/25
70/70 [==============================] - 0s 2ms/step - loss: 7.4409e-05 - val_loss: 9.6033e-05
Epoch 6/25
70/70 [==============================] - 0s 2ms/step - loss: 7.1578e-05 - val_loss: 9.2270e-05
Epoch 7/25
70/70 [==============================] - 0s 2ms/step - loss: 6.8696e-05 - val_loss: 8.9430e-05
Epoch 8/25
70/70 [==============================] - 0s 2ms/step - loss: 6.5222e-05 - val_loss: 8.7197e-05
Epoch 9/25
70/70 [==============================] - 0s 2ms/step - loss: 6.0851e-05 - val_loss: 8.3354e-05
Epoch 10/25
70/70 [==============================] - 0s 2ms/step - loss: 5.8278e-05 - val_loss: 7.7702e-05
Epoch 11/25
70/70 [==============================] - 0s 2ms/step - loss: 5.5913e-05 - val_loss: 7.8030e-05
Epoch 12/25
70/70 [==============================] - 0s 2ms/step - loss: 5.3587e-05 - val_loss: 7.4000e-05
Epoch 13/25
70/70 [==============================] - 0

Epoch 4/25
70/70 [==============================] - 0s 2ms/step - loss: 7.9327e-05 - val_loss: 1.0041e-04
Epoch 5/25
70/70 [==============================] - 0s 2ms/step - loss: 7.7127e-05 - val_loss: 9.7837e-05
Epoch 6/25
70/70 [==============================] - 0s 2ms/step - loss: 7.5547e-05 - val_loss: 9.5746e-05
Epoch 7/25
70/70 [==============================] - 0s 2ms/step - loss: 7.4340e-05 - val_loss: 1.0216e-04
Epoch 8/25
70/70 [==============================] - 0s 2ms/step - loss: 7.2883e-05 - val_loss: 9.6515e-05
Epoch 9/25
70/70 [==============================] - 0s 2ms/step - loss: 7.1438e-05 - val_loss: 9.2247e-05
Epoch 10/25
70/70 [==============================] - 0s 2ms/step - loss: 7.0604e-05 - val_loss: 1.0554e-04
Epoch 11/25
70/70 [==============================] - 0s 2ms/step - loss: 6.8983e-05 - val_loss: 9.1843e-05
Epoch 12/25
70/70 [==============================] - 0s 2ms/step - loss: 6.7597e-05 - val_loss: 8.9769e-05
Epoch 13/25
70/70 [========================

Epoch 4/25
70/70 [==============================] - 0s 2ms/step - loss: 7.2264e-05 - val_loss: 9.6851e-05
Epoch 5/25
70/70 [==============================] - 0s 2ms/step - loss: 7.0276e-05 - val_loss: 9.7492e-05
Epoch 6/25
70/70 [==============================] - 0s 2ms/step - loss: 6.8702e-05 - val_loss: 9.0682e-05
Epoch 7/25
70/70 [==============================] - 0s 2ms/step - loss: 6.6362e-05 - val_loss: 1.0289e-04
Epoch 8/25
70/70 [==============================] - 0s 2ms/step - loss: 6.3639e-05 - val_loss: 8.1687e-05
Epoch 9/25
70/70 [==============================] - 0s 2ms/step - loss: 5.8694e-05 - val_loss: 7.6205e-05
Epoch 10/25
70/70 [==============================] - 0s 2ms/step - loss: 5.3548e-05 - val_loss: 6.5873e-05
Epoch 11/25
70/70 [==============================] - 0s 2ms/step - loss: 5.1466e-05 - val_loss: 7.4976e-05
Epoch 12/25
70/70 [==============================] - 0s 2ms/step - loss: 4.5234e-05 - val_loss: 5.8868e-05
Epoch 13/25
70/70 [========================

Epoch 4/25
70/70 [==============================] - 0s 2ms/step - loss: 8.1423e-05 - val_loss: 1.0639e-04
Epoch 5/25
70/70 [==============================] - 0s 2ms/step - loss: 7.9987e-05 - val_loss: 1.0473e-04
Epoch 6/25
70/70 [==============================] - 0s 2ms/step - loss: 7.9099e-05 - val_loss: 1.0609e-04
Epoch 7/25
70/70 [==============================] - 0s 2ms/step - loss: 7.8308e-05 - val_loss: 1.0274e-04
Epoch 8/25
70/70 [==============================] - 0s 2ms/step - loss: 7.7822e-05 - val_loss: 1.0149e-04
Epoch 9/25
70/70 [==============================] - 0s 2ms/step - loss: 7.7098e-05 - val_loss: 1.0261e-04
Epoch 10/25
70/70 [==============================] - 0s 2ms/step - loss: 7.6637e-05 - val_loss: 1.0220e-04
Epoch 11/25
70/70 [==============================] - 0s 2ms/step - loss: 7.6188e-05 - val_loss: 1.0351e-04
Epoch 12/25
70/70 [==============================] - 0s 2ms/step - loss: 7.5393e-05 - val_loss: 1.0192e-04
Epoch 13/25
70/70 [========================

Epoch 4/25
70/70 [==============================] - 0s 2ms/step - loss: 6.2073e-05 - val_loss: 9.0877e-05
Epoch 5/25
70/70 [==============================] - 0s 2ms/step - loss: 5.7287e-05 - val_loss: 7.8757e-05
Epoch 6/25
70/70 [==============================] - 0s 2ms/step - loss: 5.3851e-05 - val_loss: 7.2295e-05
Epoch 7/25
70/70 [==============================] - 0s 2ms/step - loss: 5.0057e-05 - val_loss: 8.6232e-05
Epoch 8/25
70/70 [==============================] - 0s 2ms/step - loss: 4.7204e-05 - val_loss: 7.1795e-05
Epoch 9/25
70/70 [==============================] - 0s 2ms/step - loss: 4.5588e-05 - val_loss: 6.4305e-05
Epoch 10/25
70/70 [==============================] - 0s 2ms/step - loss: 4.0004e-05 - val_loss: 8.7733e-05
Epoch 11/25
70/70 [==============================] - 0s 2ms/step - loss: 3.8405e-05 - val_loss: 5.0330e-05
Epoch 12/25
70/70 [==============================] - 0s 2ms/step - loss: 3.8809e-05 - val_loss: 5.5103e-05
Epoch 13/25
70/70 [========================

In [74]:
stop = time.time() 

In [75]:
train_time_cases = stop - start
train_time_cases
# train_time = 3602.047277212143
# This took about an hour to train all the permuations of the hyperparameters

3602.047277212143

In [77]:
outcomes_cases.sort(key=lambda x: x[-1])
outcomes_cases

[[50, 25, 25, 15, 25, 25, 3.072374568091618e-07],
 [50, 25, 15, 20, 25, 15, 3.581602072699752e-07],
 [50, 25, 25, 15, 25, 20, 7.099428671608621e-07],
 [50, 25, 25, 20, 15, 15, 7.676787845412036e-07],
 [50, 25, 25, 20, 15, 20, 8.453268378616485e-07],
 [50, 25, 25, 20, 20, 25, 8.662539698889304e-07],
 [50, 25, 25, 25, 15, 25, 9.186628062707314e-07],
 [50, 25, 15, 20, 20, 20, 9.550220738674398e-07],
 [50, 25, 15, 25, 20, 25, 9.60796455728996e-07],
 [50, 25, 25, 15, 20, 15, 9.892767138808267e-07],
 [50, 25, 20, 25, 20, 15, 1.0665770560081e-06],
 [50, 25, 15, 20, 25, 20, 1.0888434189837426e-06],
 [50, 20, 25, 20, 15, 15, 1.0985437484123395e-06],
 [50, 25, 20, 15, 25, 20, 1.154611823039886e-06],
 [50, 25, 20, 20, 20, 20, 1.2280291912247776e-06],
 [50, 25, 20, 20, 15, 25, 1.2544171568151796e-06],
 [50, 20, 25, 15, 25, 20, 1.2716477613139432e-06],
 [50, 25, 20, 20, 15, 20, 1.3137420182829374e-06],
 [50, 25, 20, 25, 25, 20, 1.400978135279729e-06],
 [50, 25, 20, 15, 25, 15, 1.4129039982435643e-0

In [80]:
# Here we take the first 5 items in the sorted outcomes which means the 10 combos with the lowest loss. 
outcome_check = outcomes_cases[:5]

# These are the first five of the sorted oucome_cases - to avoid tuning again unless desired:
# outcome_check = [[50, 25, 25, 15, 25, 25, 3.072374568091618e-07],
#                  [50, 25, 15, 20, 25, 15, 3.581602072699752e-07],
#                  [50, 25, 25, 15, 25, 20, 7.099428671608621e-07],
#                  [50, 25, 25, 20, 15, 15, 7.676787845412036e-07],
#                  [50, 25, 25, 20, 15, 20, 8.453268378616485e-07]]

# Now we break up the items into seperate lists
batch_lst, epoch_lst, lstm_num_lst, dense1_lst, dense2_lst, dense3_lst = [],[],[],[],[],[]
for lst in outcome_check:
    batch_lst.append(lst[0])
    epoch_lst.append(lst[1])
    lstm_num_lst.append(lst[2])
    dense1_lst.append(lst[3])
    dense2_lst.append(lst[4])
    dense3_lst.append(lst[5])
# Now we create a dictionary which will take the mode of each hyperparameter and put it in the dictionary
cases_tuning_dict = {}
cases_tuning_dict['batch_size'] = mode(batch_lst)
cases_tuning_dict['epochs'] = mode(epoch_lst)
cases_tuning_dict['lst_num'] = mode(lstm_num_lst)
cases_tuning_dict['dense1'] = mode(dense1_lst)
cases_tuning_dict['dense2'] = mode(dense2_lst)
cases_tuning_dict['dense3'] = mode(dense3_lst)
cases_tuning_dict
# this is the result:
# cases_tuning_dict = {'batch_size': 50,
#                      'epochs': 25,
#                      'lst_num': 25,
#                      'dense1': 20,
#                      'dense2': 25,
#                      'dense3': 15}

{'batch_size': 50,
 'epochs': 25,
 'lst_num': 25,
 'dense1': 20,
 'dense2': 25,
 'dense3': 15}

In [88]:
batch_size = cases_tuning_dict['batch_size']
epochs = cases_tuning_dict['epochs']
lstm_num = cases_tuning_dict['lst_num']
dense1 = cases_tuning_dict['dense1']
dense2 = cases_tuning_dict['dense2']
dense3 = cases_tuning_dict['dense3']

model = Sequential()
model.add(LSTM(lstm_num, input_shape = (cases_train_x.shape[1], cases_train_x.shape[2])))
model.add(Dense(dense1, activation="relu"))
model.add(Dense(dense2, activation="relu"))
model.add(Dense(dense3, activation="relu"))
model.add(Dense(1))
                    
model.compile(optimizer='adam', loss='mse',)

fit=model.fit(
      cases_train_x,
      cases_train_y,
      epochs=epochs,
      validation_data=(cases_test_x, cases_test_y),
      batch_size=batch_size,)

model.evaluate(cases_test_x, cases_test_y)

Epoch 1/25
208/208 [==============================] - 0s 2ms/step - loss: 1.0393e-04 - val_loss: 8.1414e-05
Epoch 2/25
208/208 [==============================] - 0s 1ms/step - loss: 6.3104e-05 - val_loss: 7.5486e-05
Epoch 3/25
208/208 [==============================] - 0s 1ms/step - loss: 4.8751e-05 - val_loss: 6.6061e-05
Epoch 4/25
208/208 [==============================] - 0s 1ms/step - loss: 4.3579e-05 - val_loss: 1.0669e-04
Epoch 5/25
208/208 [==============================] - 0s 1ms/step - loss: 3.7140e-05 - val_loss: 4.2194e-05
Epoch 6/25
208/208 [==============================] - 0s 1ms/step - loss: 3.2307e-05 - val_loss: 3.2385e-05
Epoch 7/25
208/208 [==============================] - 0s 1ms/step - loss: 2.9756e-05 - val_loss: 4.6181e-05
Epoch 8/25
208/208 [==============================] - 0s 2ms/step - loss: 2.3286e-05 - val_loss: 2.6687e-05
Epoch 9/25
208/208 [==============================] - 0s 1ms/step - loss: 2.1239e-05 - val_loss: 2.2973e-05
Epoch 10/25
208/208 [=======

5.929758799538831e-07

In [89]:
model.save("covid_cases_model.h5")

# The models are now created, tuned, and saved